In [ ]:
import utils
import dataset

%load_ext autoreload
%autoreload 2

DEBUG = True

## Data Extraction with ArcGIS Pro

These scripts require a working installation of ArcGIS Pro with an Advanced license. As such, you must execute this notebook on a Windows machine. The Stanford School of Earth has procurred academic licenses for all Stanford Univrsity students.

Packages external to the arcgis-py3 kernel which this notebook depends on: tqdm. Remove them and refactor at your convenience.

In [ ]:
config = utils.load_config("config.yml")
if DEBUG:
    islands = ["oahu"]
else:
    islands = config["islands"]

display(config)

In [ ]:
dataset.create_shoreline_rectangles(islands, config)

In [ ]:
dataset.create_image_rectangles(islands)

In [ ]:
dataset.create_mask_rectangles(islands)